In [1]:
import torch
import mlflow
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib
from genhack.utils import plot_gpr_contourplot

In [2]:
model = mlflow.pytorch.load_model('./mlruns/0/282b102168724dea85a0222d7d102fa1/artifacts/best_ad_mean')
position = torch.tensor(np.load('data/position.npy'))

In [3]:
noise = torch.randn((3288, 6))

In [4]:
def sample(noise, position, *args, **kwargs):

    y_samples = []

    for i, z in enumerate(noise):
        gpr = model.gprs[i]
        y_mean, y_cov = gpr.predict(position.reshape(2, -1).T, return_cov=True)
        b = np.linalg.cholesky(y_cov)
        y_samples.append(y_mean + np.dot(b, z))

    return torch.tensor(np.array(y_samples))


In [5]:
def plot(day, dlat1, dlat2, dlat3, dlat4, dlat5, dlat6, dlon1, dlon2, dlon3, dlon4, dlon5, dlon6):
    
    fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
    
    position_new = position + torch.tensor([dlat1, dlat2, dlat3, dlat4, dlat5, dlat6, dlon1, dlon2, dlon3, dlon4, dlon5, dlon6])
    sst = sample(noise, position)[day]
    sst_new = sample(noise, position_new)[day]

    cmap = matplotlib.cm.get_cmap('viridis')
    norm = matplotlib.colors.Normalize(vmin=sst.min(), vmax=sst.max())

    lat, lon = position[:6], position[6:]
    for i, (lat, lon) in enumerate(zip(lat, lon)):
        ax[0].scatter(lon, lat, color=cmap(norm(sst[i])), cmap='viridis', s=100)
        ax[0].set_xlim((-15, 15))
        ax[0].set_ylim((-15, 15))

    lat, lon = position_new[:6], position_new[6:]
    for i, (lat, lon) in enumerate(zip(lat, lon)):
        ax[1].scatter(lon, lat, color=cmap(norm(sst_new[i])), cmap='viridis', s=100)
        ax[1].set_xlim((-15, 15))
        ax[1].set_ylim((-15, 15))
        
interact(plot,
         day=widgets.IntSlider(min=0, max=3288, step=1, value=0),
         dlat1=widgets.IntSlider(min=-10, max=10, step=1, value=0),
         dlat2=widgets.IntSlider(min=-10, max=10, step=1, value=0),
         dlat3=widgets.IntSlider(min=-10, max=10, step=1, value=0),
         dlat4=widgets.IntSlider(min=-10, max=10, step=1, value=0),
         dlat5=widgets.IntSlider(min=-10, max=10, step=1, value=0),
         dlat6=widgets.IntSlider(min=-10, max=10, step=1, value=0),
         dlon1=widgets.IntSlider(min=-10, max=10, step=1, value=0),
         dlon2=widgets.IntSlider(min=-10, max=10, step=1, value=0),
         dlon3=widgets.IntSlider(min=-10, max=10, step=1, value=0),
         dlon4=widgets.IntSlider(min=-10, max=10, step=1, value=0),
         dlon5=widgets.IntSlider(min=-10, max=10, step=1, value=0),
         dlon6=widgets.IntSlider(min=-10, max=10, step=1, value=0))


interactive(children=(IntSlider(value=0, description='day', max=3288), IntSlider(value=0, description='dlat1',…

<function __main__.plot(day, dlat1, dlat2, dlat3, dlat4, dlat5, dlat6, dlon1, dlon2, dlon3, dlon4, dlon5, dlon6)>

In [6]:
def plot(day):
    plot_gpr_contourplot(model, day=day, gridsize=49)
    
interact(plot, day=widgets.IntSlider(min=0, max=3288, step=1, value=10))

interactive(children=(IntSlider(value=10, description='day', max=3288), Output()), _dom_classes=('widget-inter…

<function __main__.plot(day)>

# parameter space

* different kernel for every year
* RBF
  * length = [1, 10]
* Matern
  * length = [1, 10]
  * nu = [0, 20]
* Constant
  * constant_value = [0.01, 2]

* constant * radial

# validation

* 4v2/5v1

# at the end

* try out whether crashes for different configurations in R^12

In [42]:
pd.DataFrame([sorted(np.linalg.eig(df[df.index >= f'{year}-01-01'].cov())[0]) for year in range(2008, 2017)], index=range(2008, 2017))

,0,1,2,3,4,5
2008,0.619101,0.986319,1.100103,2.576616,3.343133,8.740405
2009,0.603813,0.900279,0.980230,2.552213,3.375126,8.915066
2010,0.605333,0.858799,0.955651,2.700205,3.418625,8.152406
2011,0.567141,0.841884,1.003320,2.494177,3.344724,8.048428
2012,0.595117,0.771199,0.919864,2.160263,3.582399,8.317495
2013,0.606935,0.750858,0.895764,1.938461,3.391184,7.581810
2014,0.627194,0.636202,1.008057,2.092992,3.018795,7.676308
2015,0.425859,0.623033,0.872577,1.981311,2.945015,6.997724
2016,0.415082,0.493561,0.750464,2.008573,2.752013,7.549053
